In [ ]:
import os
import mujoco
import math
import numpy
import mediapy as media
import matplotlib.pyplot as plt

# import distutils.util
# import subprocess

In [ ]:
width = 640
height = 480

In [ ]:
xml_template = """
<mujoco>
  <option>
     <flag gravity="enable" contact="enable" />
  </option>
  <option timestep="1e-4"/>

  <default>
      <geom contype="1" conaffinity="1" condim="3" friction=".6 .3 .3" solimp=".75 .999 .1" solref=".001 1" margin="0.01" group="0"/>
  </default>
  

  <compiler angle="degree" />
    <visual><global offwidth="{width}" offheight="{height}" /></visual>    

  <worldbody>
    <light name="top" pos="0 0 1"/>
    <camera name="world" mode="fixed" pos="0 -.5 .05" axisangle="1 0 0 90"/>
    <camera name="target" mode="targetbody" target="trunk" pos="0 -.5 .05" axisangle="1 0 0 90" />

    
    <body name="floor" pos="0 0 0">
      <geom name="floor" pos="0 0 0" size="1 1 .1" type="plane" rgba="1 0.83 0.61 0.5"/>
    </body>

    <body name="trunk" pos="0 0 .1">
        <joint type="free"/>
        <geom type="box" size=".05 .05 .003" pos="0 0 0" rgba="1 0 0 1"/>
        <geom type="box" size=".005 .005 .05" pos=".05 .05 -.05" rgba="1 0 0 1"/>
        <geom type="box" size=".005 .005 .05" pos="-.05 .05 -.05" rgba="1 0 0 1"/>
        <geom type="box" size=".005 .005 .05" pos=".05 -.05 -.05" rgba="1 0 0 1"/>
        <geom type="box" size=".005 .005 .05" pos="-.05 -.05 -.05" rgba="1 0 0 1"/>
        

      <body name="leg1_l1" pos=".05 0 0">
          <joint name="leg1_j1" type="hinge" axis="0 1 0" pos="0 0 0" stiffness="{k:e}" damping="{b:e}" limited="true" range="-179 179" springref="60"/>
          <geom type="box" size=".03 .01 .001" pos=".03 0 0" rgba="1 0 0 1"/>
          <body name="leg1_l2" pos=".06 0 0">
            <joint name="leg1_j2" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179" springref="-60"/>
            <geom type="box" size=".03 .01 .001" pos=".03 0 0" rgba="1 0 0 1"/>
            <body name="leg1_f1" pos=".06 0 0">
              <joint name="leg1_j3" type="hinge" axis="0 1 0"  stiffness="{k:e}" damping="{b:e}"  limited="true" range="-179 179" springref="0"/>
              <geom type="box" size=".01 .01 .001" pos=".01 0 0" rgba="1 0 0 1"/>
            </body>
          </body>
      </body>

    </body>
  </worldbody>

   <contact>
    <exclude body1="leg1_l1" body2="leg1_l2" />
    <exclude body1="leg1_l1" body2="leg1_f1" />
    <exclude body1="leg1_l2" body2="leg1_f1" />
  </contact>

  <actuator>
    <position name="m1" joint="leg1_j1" kp="1e0"/>
    <position name="m2" joint="leg1_j2" kp="1e0"/>
  </actuator>

  <!--
  -->
</mujoco>
"""

xml = xml_template.format(width=width,height=height,k=1e-3,b=1e-4)

In [ ]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)


duration = 5  # (seconds)
framerate = 30  # (Hz)


A00 = 20 * math.pi/180
A11 = 20 * math.pi/180
f = 1
t0_1 = 0
t0_2 = -.25
b1 = 90*math.pi/180
b2 = -45*math.pi/180

def my_controller(model, data):
    t = data.time
    if t<1:
      A0=0
      A1=0
    elif t<2:
      A0 = A00*(t-1)/(2-1)
      A1 = A11*(t-1)/(2-1)
    else:
      A0 = A00
      A1 = A11
    j1 = A0*math.sin(2*math.pi*(f*t-t0_1))+b1
    j2 = A1*math.sin(2*math.pi*(f*t-t0_2))+b2
    # data.ctrl = [j1]
    data.ctrl = [j1,j2]
    return

mujoco.set_mjcb_control(my_controller)

frames = []
mujoco.mj_resetData(model, data)
data.qpos[0:3] = numpy.array([0,0,.25])
data.qpos[7:] = numpy.array([60,-60,-60])* math.pi/180
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data,camera='target')
    pixels = renderer.render()
    frames.append(pixels)

# Simulate and display video.
media.show_video(frames, fps=framerate,width=width,height=height)

In [ ]:
data.qpos

In [ ]:
data.tendon